In [1]:
#Imports
import pandas as pd
from datetime import datetime
import pickle
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [17]:
#Load and Clean Carcass Data
carcassDf = pd.read_excel("../datafiles/GRADE Form 11-8-21 HEARTBRAND-.xlsx", sheet_name="Individual Grades ", engine='openpyxl')
carcassDf = carcassDf.iloc[5:].reset_index(drop=True)
carcassDf = carcassDf.dropna(axis=1, how='all')
carcassDf.columns = carcassDf.iloc[0]
carcassDf = carcassDf[1:].reset_index(drop=True) 
carcassDf.columns.values[4] = "Tattoo"
carcassDf['Tattoo'] = carcassDf['Tattoo'].astype(str)
carcassDf = carcassDf.rename(columns={'Weight': 'Carcass_Weight'})
carcassDf = carcassDf.dropna(how='all')
carcassDf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   nan             289 non-null    float64
 1   Carcass #       285 non-null    object 
 2   TAGS            201 non-null    object 
 3   nan             285 non-null    object 
 4   Tattoo          379 non-null    object 
 5   Sex             285 non-null    object 
 6   Color           285 non-null    object 
 7   Carcass_Weight  285 non-null    object 
 8   Quality         285 non-null    object 
 9   Yield           286 non-null    object 
 10  nan             285 non-null    object 
 11  Member ID       277 non-null    object 
 12  Member Name     277 non-null    object 
 13  nan             1 non-null      object 
dtypes: float64(1), object(13)
memory usage: 41.6+ KB


In [5]:
#Load and Clean CattleMax data
def clean_and_modify_CattlemaxDfs(df):
    # st.write("In clean_and_modify_CattlemaxDfs")
    # st.dataframe(df)
    today = datetime.today()
    df = df.rename(columns={
        'Calving Ease Direct EPD': 'CED',
        'Calving Ease Direct Acc': 'CED Acc',
        'Birth Weight EPD': 'BW',
        'Birth Weight Acc': 'BW Acc',
        'Weaning Weight EPD': 'WW',
        'Weaning Weight Acc': 'WW Acc',
        'Yearling Weight EPD': 'YW',
        'Yearling Weight Acc': 'YW Acc',
        'Milk EPD': 'MK',
        'Milk Acc': 'MK Acc',
        'Total Maternal EPD': 'TM'
    })
    df = df.dropna(axis=1, how='all')
    df['Growth'] = (- 0.301 * df['BW'] ) + \
                   ( 0.039 * df['WW']) + \
                   (1.098 * df['YW'] ) - 6.815
    df['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce')
    df['Year_Born'] = df['Date of Birth'].dt.year
    df['Age'] = (pd.to_datetime('today').year) - df['Date of Birth'].dt.year
    df['Tattoo'] = df['Tattoo'].astype(str)
    def assign_designation(row):
        if pd.isnull(row['Year_Born']):
            return 'Unknown'  # Handle cases where Year_Born is missing
        
        age_in_years = today.year - int(row['Year_Born'])
        
        if row['Type or Sex'] == 'B' and age_in_years >= 2:
            return 'Bull'
        elif row['Type or Sex'] == 'C' and age_in_years >= 2:
            return 'Dam'
        elif row['Type or Sex'] in ['B', 'C'] and age_in_years < 2:
            return 'Non-Parent'
        elif row['Type or Sex'] == 'S':
            return 'Steer'
        
        return 'Unknown'  # Default for rows that don't match any condition
    df['Designation'] = df.apply(assign_designation, axis=1)
    
    return(df)
cattleMaxDf = pd.read_csv("../datafiles/ArtesianFULL_cattlemax_herdfile_291.csv",index_col=False)

cattleMaxDf = clean_and_modify_CattlemaxDfs(cattleMaxDf)
cattleMaxDf.info()
#If you now need to go get the pedigree for this herd file you can use the following code
# cattleMaxDf["Registration Number"].to_csv("Registration_numbers.csv", index=False, header=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3314 entries, 0 to 3313
Data columns (total 69 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Status/Disposal           3314 non-null   int64         
 1   Brand                     2670 non-null   object        
 2   Brand Location            2298 non-null   object        
 3   Tattoo                    3314 non-null   object        
 4   Tattoo Location           928 non-null    object        
 5   Name                      3308 non-null   object        
 6   Registration Number       3314 non-null   object        
 7   Breeder                   2694 non-null   object        
 8   Breeder Member Number     2694 non-null   float64       
 9   Owner                     2802 non-null   object        
 10  Owner Member Number       2802 non-null   float64       
 11  Date of Birth             3299 non-null   datetime64[ns]
 12  Birth Method        

In [15]:
#Load and Clean 5 Gen Pedigree Data

pedigreeDf = pd.read_csv("../datafiles/Artesian_5Gen_Pedigree_ALL.csv",index_col=False)
pedigreeDf = pedigreeDf.rename(columns={'animal_registration': 'Registration Number'})

pedigreeDf.head()

,Registration Number,phn,animal_name,sire,sire_phn,sire_name,dam,dam_phn,dam_name,sire_sire,sire_sire_phn,sire_sire_name,sire_dam,sire_dam_phn,sire_dam_name,dam_sire,dam_sire_phn,dam_sire_name,dam_dam,dam_dam_phn,dam_dam_name,sire_sire_sire,sire_sire_sire_phn,sire_sire_sire_name,sire_sire_dam,sire_sire_dam_phn,sire_sire_dam_name,sire_dam_sire,sire_dam_sire_phn,sire_dam_sire_name,sire_dam_dam,sire_dam_dam_phn,sire_dam_dam_name,dam_sire_sire,dam_sire_sire_phn,dam_sire_sire_name,dam_sire_dam,dam_sire_dam_phn,dam_sire_dam_name,dam_dam_sire,dam_dam_sire_phn,dam_dam_sire_name,dam_dam_dam,dam_dam_dam_phn,dam_dam_dam_name,sire_sire_sire_sire,sire_sire_sire_sire_phn,sire_sire_sire_sire_name,sire_sire_sire_dam,sire_sire_sire_dam_phn,sire_sire_sire_dam_name,sire_sire_dam_sire,sire_sire_dam_sire_phn,sire_sire_dam_sire_name,sire_sire_dam_dam,sire_sire_dam_dam_phn,sire_sire_dam_dam_name,dam_sire_sire_sire,dam_sire_sire_sire_phn,dam_sire_sire_sire_name,dam_sire_sire_dam,dam_sire_sire_dam_phn,dam_sire_sire_dam_name,dam_sire_dam_sire,dam_sire_dam_sire_phn,dam_sire_dam_sire_name,dam_sire_dam_dam,dam_sire_dam_dam_phn,dam_sire_dam_dam_name,sire_dam_sire_sire,sire_dam_sire_sire_phn,sire_dam_sire_sire_name,sire_dam_sire_dam,sire_dam_sire_dam_phn,sire_dam_sire_dam_name,sire_dam_dam_sire,sire_dam_dam_sire_phn,sire_dam_dam_sire_name,sire_dam_dam_dam,sire_dam_dam_dam_phn,sire_dam_dam_dam_name,dam_dam_sire_sire,dam_dam_sire_sire_phn,dam_dam_sire_sire_name,dam_dam_sire_dam,dam_dam_sire_dam_phn,dam_dam_sire_dam_name,dam_dam_dam_sire,dam_dam_dam_sire_phn,dam_dam_dam_sire_name,dam_dam_dam_dam,dam_dam_dam_dam_phn,dam_dam_dam_dam_name
0,X93971,932,932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1T110153,7019,7019-722,AF36497,1603Z,HEARTBRAND 1603Z,X93942,722,722,AF5638,S5204S,HEARTBRAND S5204S,AF8267,V6825U,HEARTBRAND V6825U,NaN,NaN,NaN,NaN,NaN,NaN,AF110,HB 457,SHIGEMARU,AF138,B0905J,HEARTBRAND B0905J,AF109,HB 459,TAMAMARU,AF1872,T1610P,HEARTBRAND T1610P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AJ114,TOK 27,DAI-NI 2 SHIGENAMI TOKKYU 27,AJ120,TOK 3357,DAI-YON 4 AKIBARE TOKKYU 3357,AF114,HB502E,BIG AL,AF106,FJ357B,FUYUKO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AJ125,TOK 96,DAI-GO 5 TAMANAMI TOKKYU 96,AF101,FJ461Z,DAI-HACHI 8 MARUNAMI,AF109,HB 459,TAMAMARU,AF161,B0951J,HEARTBRAND B0951J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1T110143,7087,7087-705A,AF36497,1603Z,HEARTBRAND 1603Z,X93932,705,705,AF5638,S5204S,HEARTBRAND S5204S,AF8267,V6825U,HEARTBRAND V6825U,NaN,NaN,NaN,NaN,NaN,NaN,AF110,HB 457,SHIGEMARU,AF138,B0905J,HEARTBRAND B0905J,AF109,HB 459,TAMAMARU,AF1872,T1610P,HEARTBRAND T1610P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AJ114,TOK 27,DAI-NI 2 SHIGENAMI TOKKYU 27,AJ120,TOK 3357,DAI-YON 4 AKIBARE TOKKYU 3357,AF114,HB502E,BIG AL,AF106,FJ357B,FUYUKO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AJ125,TOK 96,DAI-GO 5 TAMANAMI TOKKYU 96,AF101,FJ461Z,DAI-HACHI 8 MARUNAMI,AF109,HB 459,TAMAMARU,AF161,B0951J,HEARTBRAND B0951J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EXX1,NaN,CROSSBREED BULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,X93914,448,448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [22]:
#Merge CattleMax and Pedigree Dfs first given they are the most comprehensive: 
merged_df = cattleMaxDf.merge(pedigreeDf, on="Registration Number", how="left", indicator=True)
print(f"Length of CattleMaxDf is {len(cattleMaxDf)}")
print(f"Length of pedigreeDf is {len(pedigreeDf)}")
no_match_count = merged_df[merged_df['_merge'] == 'left_only'].shape[0]
print(f"Number of instances in cattleMaxDf without a match in pedigreeDf: {no_match_count}")

Length of CattleMaxDf is 3314
Length of pedigreeDf is 3314
Number of instances in cattleMaxDf without a match in pedigreeDf: 0


In [26]:
merged_df.columns

Index(['Status/Disposal', 'Brand', 'Brand Location', 'Tattoo',
       'Tattoo Location', 'Name', 'Registration Number', 'Breeder',
       'Breeder Member Number', 'Owner',
       ...
       'dam_dam_sire_dam', 'dam_dam_sire_dam_phn', 'dam_dam_sire_dam_name',
       'dam_dam_dam_sire', 'dam_dam_dam_sire_phn', 'dam_dam_dam_sire_name',
       'dam_dam_dam_dam', 'dam_dam_dam_dam_phn', 'dam_dam_dam_dam_name',
       '_merge'],
      dtype='object', length=162)

In [27]:
#Now merge in the carcass data to the merged Df
# Merge with indicator to identify unmatched rows
superDf = merged_df.merge(
    carcassDf[['Tattoo', 'Carcass #', 'TAGS', 'Carcass_Weight', 'Quality', 'Yield', 'Member ID', 'Member Name']],
    on='Tattoo',
    how='left',
    indicator="carcass_merge"
)

# Filter rows in carcassDf that did not have a match in cattleMaxDf
unmatched_rows = superDf[superDf['carcass_merge'] == 'right_only']

# Get the count of unmatched rows
num_unmatched_rows = unmatched_rows.shape[0]

# Print total rows and unmatched counts
print(f"Total number of rows in carcassDf: {carcassDf.shape[0]}")
print(f"Number of rows in carcassDf not in cattleMaxDf: {num_unmatched_rows}")

Total number of rows in carcassDf: 379
Number of rows in carcassDf not in cattleMaxDf: 0
